# IMPORT

In [31]:
import requests
from bs4 import BeautifulSoup
import os

# FUNCTIONS

In [32]:
def scraper(ufficio,ente,anno,numero,natura):
    """
    ufficio è l'id che si trova nel file uffici_finanziari (in code)
    ente è l'id da ricavare da enti.json   (esempio 001 )
    numero da file input (rg / decreto / etc )
    anno anno con 4 cifre da file input
    natura  DI  AV  /  EM  OR
    """


    session = requests.Session()
    initial_url = 'https://www1.agenziaentrate.gov.it/servizi/tassazioneattigiudiziari/registrazione.htm'
    session.get(initial_url) 

    url1 = 'https://www1.agenziaentrate.gov.it/servizi/tassazioneattigiudiziari/registrazione.htm'
    params1 = {
        'action': 'scegliufficio',
        'ufficio': ufficio,
    }
    headers = {
        'Connection': 'keep-alive',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept': '*/*',
        'Content-Type': 'application/x-www-form-urlencoded',
    }
    body1 = {
        'ufficio': ufficio,
    }
    response1 = session.post(url1, headers=headers, params=params1, data=body1)
    
    if response1:
        url2 = 'https://www1.agenziaentrate.gov.it/servizi/tassazioneattigiudiziari/registrazione.htm'
        params2 = {
            'action': 'scegliente',
        }
        body2 = {
            'ente': ente,
            'anno': anno,
            'numero': numero,
            'natura' : natura,
            'snumero' : ""
        }
        response2 = session.post(url2, headers=headers, params=params2, data=body2)
        if response2 :
            soup2 = BeautifulSoup(response2.text, 'html.parser')
            if 'non trovato' in response2.text:
                rep = str(numero) + ' - ' + str(anno)
                return [ufficio, natura, ente, rep, '-', '-', '-','no']
            elif 'selezionare' in response2.text.lower():
                rep = str(numero) + ' - ' + str(anno)
                return [ufficio, natura, ente, rep, '-', '-', '-','ente mancante / possibile altro errore']
            else :
                compila = 'no'
                tabella = soup2.find('div', class_='panel-body')

                if  tabella.find('span', string="Atto tassato dall'ufficio di: "): 
                    office = tabella.find('span', string="Atto tassato dall'ufficio di: ").find_next_sibling('span').get_text(strip=True)
                else :
                    office = '-'   

                if tabella.find('span', string="Tipologia atto: "):
                    atto = tabella.find('span', string="Tipologia atto: ").find_next_sibling('span').get_text(strip=True)
                else :
                    atto = '-'  

                if tabella.find('span', string="Emesso da: "):
                    emesso_da = tabella.find('span', string="Emesso da: ").find_next_sibling('span').get_text(strip=True)
                else :
                    emesso_da = '-'   

                if tabella.find('span', string="n. Repertorio - anno: "):
                    repertorio = tabella.find('span', string="n. Repertorio - anno: ").find_next_sibling('span').get_text(strip=True)
                else :
                    repertorio = '-'  

                if tabella.find('span', string="Attore: "):
                    attore = tabella.find('span', string="Attore: ").find_next_sibling('span').get_text(strip=True)
                else:
                    attore='-'

                if tabella.find('span', string="Convenuto: "):
                    convenuto = tabella.find('span', string="Convenuto: ").find_next_sibling('span').get_text(strip=True)
                else:
                    convenuto ='-'

                if tabella.find('span', string="Stato dell'atto: "):
                    stato = tabella.find('span', string="Stato dell'atto: ").find_next_sibling('span').get_text(strip=True)
                else:
                    stato= '-'

                if 'compilaf24.php' in response2.text: 
                    compila = 'si'

                return [office,atto,emesso_da,repertorio,attore,convenuto,stato,compila]
        else :
            rep = str(numero) + ' - ' + str(anno)
            return [ufficio, natura, ente, rep, '-', '-', '-','no']

    else:
       return 'ufficio non trovato'   

In [33]:
list_natura_monitoria = ['DI','AV']
list_natura_ppt = ['EM','OR']

def tipo_scraper(contratto,debitore, tipo, ufficio, lista_enti, anno, numero,row_number):
    print(row_number)
    results = []
    try:
        if tipo == 'monitoria':
            natura_list = list_natura_monitoria
        elif tipo == 'ppt':
            natura_list = list_natura_ppt
        else:
            raise ValueError("Invalid tipo value")

        for natura in natura_list:
            for ente in lista_enti:
                result=''
                result = scraper(ufficio, ente, anno, numero, natura)
                if isinstance(result, list):
                    if len(result) == 8:
                        results.append({
                            "contratto":contratto,
                            "debitore":debitore,
                            'ufficio': result[0],
                            'atto': result[1],
                            'emesso da': result[2],
                            'repertorio': result[3],
                            'attore': result[4],
                            'convenuto': result[5],
                            'stato': result[6],
                            "compila":result[7]
                        })
                    else:
                        rep = str(numero) + ' - ' + str(anno)
                        results.append({
                            "contratto":contratto,
                            "debitore":debitore,
                            'ufficio': ufficio,
                            'natura': natura,
                            'ente': ente,
                            'repertorio': rep,
                            'attore': '-',
                            'convenuto': '-',
                            'stato': '-',
                            "compila":'no'
                        })
                else:
                    print(result)
    except Exception as e:
        print(f'Riga con errore: {row_number} , ente : {ente} , natura : {natura} , debitore : {debitore}')
        print(f"Error occurred: {e}")
    return results

# tipo_scraper('1','de','ppt','TQE',['001'],20,2,1)

# PULIZIA INPUT

In [34]:
from Pulizia import clean , split_and_expand
import pandas as pd
import json
from datetime import date
import numpy as np
import re
today = date.today()


In [35]:
uffici = pd.read_excel('Resources/uffici_finanziari.xlsx')
uffici = uffici.iloc[1:]
uffici['office'] = uffici['office'].str.lower()
uffici_ut = uffici[uffici['office'].str.contains('ut')]

In [36]:
input0 = pd.read_excel('Input/Raw/perimetro_03_06.xlsx')

In [37]:
input = input0.copy()

In [38]:
len(input)

2350

In [39]:
n_columns = input.filter(regex='^N')
contains_dash = n_columns.applymap(lambda x: '-' in str(x) and '/' not in str(x))
n_columns = n_columns.applymap(lambda x: x.replace('-', '/') if '-' in str(x) else x)
input[n_columns.columns] = n_columns

In [40]:
input = input[input['N. RG'].str.contains('/', na=True) | input['N. RG'].isna()]
input = input[input['N. Decreto'].str.contains('/', na=True) | input['N. Decreto'].isna()]
input = input[input['N.R.G.E PPT'].str.contains('/', na=True) | input['N.R.G.E PPT'].isna()]
input = input[input['N° Repertorio'].str.contains('/', na=True) | input['N° Repertorio'].isna()]
input = input[input['N° Rep PPT'].str.contains('/', na=True) | input['N° Rep PPT'].isna()]
input = input[input['N. Cronologico'].str.contains('/', na=True) | input['N. Cronologico'].isna()]

In [41]:
input = input.drop_duplicates()
input = clean(input)

Split did not produce the expected number of columns for some rows.


In [42]:
len(input) # non deve diminuire

2350

In [43]:
mapping = {'albenga':'savona',
    'chiavenna':'sondrio',
    'cossato':'biella',
    'macomer':'nuoro',
    'isili':'nuoro',
    'chiaromonte': 'lagonegro',
    'orzinuovi' : 'brescia',
    'ovada': 'acquiterme',
    'pavullo nel frignano':'modena',
    'ponte san pietro':'bergamo',
    'romano di lombardia':'bergamo',
    'san benedetto del tronto':'ascoli piceno',
    'teano':'caserta',
    'venezia 2 - mestre' : 'venezia',
    'venosa':'potenza',
    'vimercate':'monza',
    'acerra' : 'casoria',
    'torre annunziata':'napoli',
    'napoli nord in aversa':'napoli',
    'napoli nord':'napoli',
    'san giorgio ionico':'taranto',
    "rometta" :'barcellona pozzo di gotto',
    'taurianova':'palmi',
    'putignano':'gioia del colle',
    'dolo':'venezia',
    'ugento':'casarano',
    'grumello del monte':'bergamo',
    'todi':'perugia',
    'gragnano':'castellammare di stabia',
    'tribunale  di vicenza':'vicenza',
    'marigliano':'nola',
    "reggio nell'emilia": 'reggio emilia',
    'montoro':'avellino',
    'nocera inferiore':'pagani',
    "chieti - sezione distaccata di ortona":'chieti',
    'la maddalena':'olbia',
    'san giovanni in fiore':'cosenza',
    'poggio mirteto':'rieti',
    'volterra':'pontedera',
    'tribunale della spezia':'la spezia',
    'arcidosso':'grosseto',
    'martina franca':'taranto',
    'naso':'messina',
    'scalea':'paola',
    'cariati':'rossano',
    'montecorvino rovella':'salerno',
    "sant'anastasia":'napoli',
    'forlì':"forli'"
    }

input['Foro'] = (input['Foro']
                    .str.lower()
                    .replace(mapping, regex=True)
                    .str.strip()
                   )
# input
input = input.drop_duplicates()

In [44]:
def find_code_for_foro(df1, df2):
    # Initialize an empty list to store the results
    results = []

    # Iterate over each row in df2
    for index2, row2 in df2.iterrows():
        foro = row2['Foro']
        
        # Find matching rows in df1
        matching_rows = df1[df1['office'].str.contains(foro)]

        # Add matching codes to the results list
        for index1, row1 in matching_rows.iterrows():
            results.append({
                'code_office': row1['code'],
                'office': row1['office'],
                'Foro': row2['Foro']
            })

    # Convert results to a DataFrame
    result_df = pd.DataFrame(results)
    return result_df
result_df = find_code_for_foro(uffici_ut, input)
result_df = result_df.drop_duplicates()

In [45]:
input = pd.merge(input, result_df, on='Foro', how='left')
input = (input
        .drop_duplicates())
input['Tribunale/Gdp'] = (input['Tribunale/Gdp']
                          .str.lower()
                          .str.strip()
)

input = input.drop_duplicates()

Se questo non è [], sistemare il mapping

In [46]:
input[input['code_office'].isna()]['Foro'].drop_duplicates()

Series([], Name: Foro, dtype: object)

In [47]:
input['Debitore'] = (input['Debitore']
                     .str.replace('-', ' ')
                     .str.replace(r'\s+', ' ', regex=True)
                     .str.strip()
                    )


In [48]:
with open('Resources/enti_completi_ut.json', 'r') as file:
    enti_ut = json.load(file)

enti_decoded = {}

for key, values in enti_ut.items():
    tribunale = [item.split(" - ")[0] for item in values if "TRIB" in item.upper()]
    giudice = [item.split(" - ")[0] for item in values if "GIUD" in item.upper()]
    enti_decoded[key] = {"tribunale": tribunale, "giudice": giudice}
# enti_decoded    

In [49]:
def get_codes(row):
    office = row['office']
    type_ = row['Tribunale/Gdp']
    if office in enti_decoded and type_ in enti_decoded[office]:
        return enti_decoded[office][type_]
    else:
        return []

# Apply the function to each row and create a new column 'codes'
input['code_ente'] = input.apply(get_codes, axis=1)


In [50]:
len(input)

3000

In [51]:
# Convert the 'code_ente' lists to strings
input['code_ente'] = input['code_ente'].apply(lambda x: ','.join(x))


potential_value_vars = [
    'nr.decreto', 'year.decreto', 'nr.rg', 'year.rg',
    'nr.repertorio', 'year.repertorio', 'nr.crono', 'year.crono',
    'nr.rgeppt', 'year.rgeppt', 'nr.repppt', 'year.repppt'
]
value_vars = [col for col in potential_value_vars if col in input.columns]
# Melt the DataFrame to 'pivot longer'
df_long = pd.melt(input, 
                  id_vars=['NR_Contratto', 'Debitore','Tribunale/Gdp','Foro','Cliente','tipo_ricerca','code_office','office','code_ente'], 
                  value_vars=value_vars, 
                  var_name='type', 
                  value_name='value')

# Split the 'type' column into 'attribute' and 'category'
df_long[['attribute', 'category']] = df_long['type'].str.split('.', expand=True)

# Pivot the DataFrame to get the desired format
input = df_long.pivot_table(index=['NR_Contratto', 'Debitore','Tribunale/Gdp','Foro','Cliente','tipo_ricerca','code_office','office','code_ente','category'], 
                               columns='attribute', 
                               values='value', 
                               aggfunc='first').reset_index()

# Flatten the columns
input.columns.name = None
input = input.reset_index(drop=True)

# Convert the 'code_ente' column back into a list
input['code_ente'] = input['code_ente'].apply(lambda x: x.split(',') if isinstance(x, str) else x)


In [52]:
input = input.drop(columns={'category'})
input['code_ente'] = input['code_ente'].apply(tuple)
input = input.drop_duplicates()
input['code_ente'] = input['code_ente'].apply(list)

In [53]:
len(input)

6449

se ci sono risultati manda a daniele

In [54]:
x = input[(input['year']!='')]
x['year'] = x['year'].astype(int)
x[(x['year']>2024) | (x['year']<1900)]



C:\Users\claudia.esposito\AppData\Local\Temp\ipykernel_4968\439951867.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['year'] = x['year'].astype(int)


,NR_Contratto,Debitore,Tribunale/Gdp,Foro,Cliente,tipo_ricerca,code_office,office,code_ente,nr,year
31,AA1700000339_04022021,AOUACHE HASSAN,giudice,mantova,A2A,monitoria,TNH,mantova ut dpmn,"[004, 006, 007]",1631,2921
5334,UB13213055_15301327800,PRESUTTI GIULIANO,tribunale,ascoli piceno,Sirio,monitoria,TQE,ascoli piceno ut dpap,[001],2061,202
6017,UBS380208,LICITRA GIULIA,giudice,ancona,Sirio,monitoria,TQD,ancona ut dpan,"[005, 006]",1177,223


In [55]:
input.to_excel(f'Input/Cleaned/{today}_input_v1.xlsx',index=False)

In [56]:
#4498   ??? 
len(input[['NR_Contratto','nr','year']].drop_duplicates())

4955

In [57]:
input_vecchio = pd.read_excel('Input/Cleaned/2024_06_03_input_v1.xlsx',dtype={'nr':str,'year':str})

input_vecchio['Debitore'] = (input_vecchio['Debitore']
                     .str.replace('-', ' ')
                     .str.replace(r'\s+', ' ', regex=True)
                     .str.strip()
                    )


In [58]:
input['code_ente'] = input['code_ente'].astype(str)
input_vecchio['code_ente'] = input_vecchio['code_ente'].astype(str)

In [59]:
diff_df = input.merge(input_vecchio, how='outer', indicator=True)
# Filter the rows that are different
diff_rows = diff_df[diff_df['_merge'] == 'left_only']
diff_rows['code_ente'] = diff_rows['code_ente'].apply(eval)
diff_rows

C:\Users\claudia.esposito\AppData\Local\Temp\ipykernel_4968\946771423.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diff_rows['code_ente'] = diff_rows['code_ente'].apply(eval)


,NR_Contratto,Debitore,Tribunale/Gdp,Foro,Cliente,tipo_ricerca,code_office,office,code_ente,nr,year,_merge
1,AA1200001768_06112018,SGOBBA IVAN,giudice,brescia,A2A,monitoria,TM4,brescia ut dpbs,"[003, 005, 006, 009]",1305,2020,left_only
4,AA1200001768_06112018,SGOBBA IVAN,giudice,brescia,A2A,monitoria,TMJ,brescia 1 ut dpbs,"[003, 005, 006, 009]",1305,2020,left_only
7,AA1200002409_06112018,TAVARES FARIAS CARLOS ERIK,giudice,brescia,A2A,monitoria,TM4,brescia ut dpbs,"[003, 005, 006, 009]",1256,2020,left_only
10,AA1200002409_06112018,TAVARES FARIAS CARLOS ERIK,giudice,brescia,A2A,monitoria,TMJ,brescia 1 ut dpbs,"[003, 005, 006, 009]",1256,2020,left_only
13,AA1200003174_06112018,TORTORA VINCENZA,giudice,brescia,A2A,monitoria,TM4,brescia ut dpbs,"[003, 005, 006, 009]",1290,2020,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...
6439,XUB385075,BUCCIONE PIERO,tribunale,chieti,Sirio,monitoria,TAH,chieti ut dpch,[001],365,2023,left_only
6441,XUB385803,STOP CAFFE' SNC DI GENNARELLI GENNARO E GUARAG...,tribunale,civitavecchia,Sirio,monitoria,TJ3,civitavecchia ut dprm2,[001],226,2023,left_only
6443,XUB385803,STOP CAFFE' SNC DI GENNARELLI GENNARO E GUARAG...,tribunale,civitavecchia,Sirio,monitoria,TJF,civitavecchia - ante 01.02.2015 ut dprm1,"[001, 004, 005, 006, 007, 011]",226,2023,left_only
6445,XUB386298,PESCE MAURO,tribunale,pescara,Sirio,monitoria,TAU,pescara ut dppe,"[001, 005, 006]",1975,2021,left_only


# SCRAPER

Modificare new_df, 500 per volta

In [80]:
a=500
b=1000
# b = "fine"

new_df = diff_rows[a:b]

# new_df = input[a:b]
results =( new_df 
    .apply(lambda row: tipo_scraper(contratto=row['NR_Contratto'],
                                    debitore=row['Debitore'],
                                    tipo= row['tipo_ricerca'],
                                    ufficio= row['code_office'],
                                    lista_enti=row['code_ente'], 
                                    anno=row['year'],
                                    numero= row['nr'],
                                    row_number=row.name
                                    ),
                                    axis=1)
)

1026
1028
1030
1032
1034
1036
1038
1040
1042
1044
1046
1048
1050
1052
1054
1056
1058
1060
1062
1064
1066
1068
1070
1072
1074
1076
1078
1080
1082
1084
1086
1088
1090
1091
1093
1095
1097
1099
1101
1103
1105
1107
1109
1111
1113
1115
1117
1119
1122
1124
1126
1129
1131
1133
1136
1139
1142
1145
1147
1149
1151
1153
1155
1157
1160
1163
1165
1167
1169
1172
1175
1178
1180
1182
1185
1187
1189
1192
1194
1196
1198
1200
1202
1204
1206
1209
1211
1214
1217
1220
1222
1224
1226
1228
1231
1233
1236
1239
1242
1245
1247
1250
1252
1254
1256
1258
1260
1262
1264
1266
1268
1270
1272
1274
1276
1278
1280
1282
1284
1286
1288
1290
1292
1294
1296
1298
1299
1300
1302
1304
1306
1308
1310
1312
1314
1316
1318
1320
1322
1324
1326
1328
1330
1332
1334
1336
1338
1340
1342
1344
1346
1348
1350
1352
1354
1356
1358
1360
1362
1364
1366
1368
1370
1372
1374
1376
1378
1381
1383
1385
1387
1389
1391
1394
1398
1401
1403
1405
1407
1409
1411
1413
1415
1417
1419
1421
1423
1425
1428
1431
1434
1436
1438
1440
1442
1444
1446
1448
1450
1452


# PULIZIA RISULTATI

In [61]:
filtered_data = [item for item in results if item != []]
flattened_results = [item for sublist in filtered_data for item in (sublist if isinstance(sublist, list) else [sublist])]
df = pd.DataFrame(flattened_results)
df = df.drop_duplicates()

In [62]:
cliente = input0[['NR_Contratto','Cliente']]
df = pd.merge(df,cliente,how='left', left_on='contratto',right_on='NR_Contratto' )
df.drop(columns=['NR_Contratto'], inplace=True)
df['Cliente'] = df['Cliente'].str.upper()
df = df.drop_duplicates()

# Check visivi

In [63]:
# Righe con risultati
df[(df['attore'] != '-') | (df['convenuto'] != '-') ]

,contratto,debitore,ufficio,atto,emesso da,repertorio,attore,convenuto,stato,compila,Cliente
96,AA1700000444_04022021,AMATO MORENO,MANTOVA UT DPMN,DECRETO INGIUNTIVO n. 1450 del 2021,GIUDICE DI PACE-MANTOVA,637 - 2021,APRICA S.P.A.,AMATO,ATTO IN FASE DI TASSAZIONE,no,A2A
186,AA1700000803_04022021,UMORU FLORENCE BUKKY,MANTOVA UT DPMN,DECRETO INGIUNTIVO n. 1856 del 2021,GIUDICE DI PACE-MANTOVA,664 - 2021,APRICA S.P.A.,UMORU,ATTO IN FASE DI TASSAZIONE,no,A2A
204,AA1700000871_04022021,BELLOTTI FABRIZIO,MANTOVA UT DPMN,DECRETO INGIUNTIVO n. 1846 del 2021,GIUDICE DI PACE-MANTOVA,638 - 2021,APRICA S.P.A.,BELLOTTI,ATTO IN FASE DI TASSAZIONE,no,A2A
210,AA1700000896_04022021,VISENTINI SAMANTA,MANTOVA UT DPMN,DECRETO INGIUNTIVO n. 62 del 2022,GIUDICE DI PACE-MANTOVA,96 - 2022,APRICA SPA,VISENTINI,ATTO IN FASE DI TASSAZIONE,no,A2A
216,AA1700000914_04022021,CHIARELLI MARIA PIA,MANTOVA UT DPMN,DECRETO INGIUNTIVO n. 1951 del 2021,GIUDICE DI PACE-MANTOVA,666 - 2021,APRICA S.P.A.,CHIARELLI,ATTO IN FASE DI TASSAZIONE,no,A2A
...,...,...,...,...,...,...,...,...,...,...,...
5294,AZCO20190204980,RENZETTI GISBERTO,PALERMO 2 UT DPPA,DECRETO INGIUNTIVO n. 1938 del 2021,GIUDICE DI PACE DI PALERMO,2429 - 2021,POS SRL,GISBERTO RENZETTI,ATTO REGISTRATO,no,AZETA
5326,AZCO20190205016,LEDDA ENRICO,ORISTANO UT DPOR,DECRETO INGIUNTIVO n. 167 del 2021,TRIBUNALE DI ORISTANO,284 - 2021,A-ZETA S.R.L.,LEDDA,ATTO REGISTRATO,no,AZETA
5363,AZCO20190205058,ROSSI MASSIMO,RIETI UT DPRI,DECRETO INGIUNTIVO n. 58 del 2021,GIUDICE DI PACE DI P. MIRTETO,34 - 2021,POS SRL,-,ATTO REGISTRATO,no,AZETA
5374,AZCO20190205059,BODNAR AURELIYA,MILANO 1 UT DPMI1,DECRETO INGIUNTIVO n. 2089 del 2022,TRIB. CIVILE ORDINARIO,10637 - 2022,POS SRL,-,ATTO IN FASE DI TASSAZIONE,no,AZETA


In [64]:
# Guardare i nomi in format 'name +1'
df[(df['attore'].str.contains('\+')) | (df['convenuto'].str.contains('\+')) ]

,contratto,debitore,ufficio,atto,emesso da,repertorio,attore,convenuto,stato,compila,Cliente
2348,AKCO400874222.001,TRONCI PAOLO ROCCO,CASARANO UT DPLE,DECRETO INGIUNTIVO n. 34 del 2017,GIUDICE DI PACE DI GALLIPOLI,25/17 - 2017,AT NPL'S S.P.A.,GRECO VINCENZO +1,ATTO REGISTRATO,no,AK NORDIC
3682,AKS008439548,GARBUGLIA GABRIELLA,TREVISO UT DPTV,DECRETO INGIUNTIVO n. 2851 del 2023,GIUDICE DI PACE DI TREVISO,933/24 - 2023,AK NORDIC AB,GARBUGLIA+MOINO,IN ATTESA DI PAGAMENTO,si,AK NORDIC
4064,AZAL20200000394,BOCAJ KRESHNIK,LECCE UT DPLE,DECRETO INGIUNTIVO n. 49 del 2021,TRIBUNALE DI LECCE,154-21 - 2021,AR HOLDING S.P.A.,BOCAJ KRESHNIK + 1,ATTO REGISTRATO,no,AZETA


# SALVATAGGIO OUTPUT

In [65]:
df['attore'] = (df['attore']
                .str.replace('.', '', regex=False)
                .str.replace('-', '', regex=False)
                .str.strip()
                )
df['convenuto'] = (df['convenuto']
                .str.replace('.', '', regex=False)
                .str.replace('-', '', regex=False)
                .str.strip()
                )
df['debitore'] = (df['debitore']
                .str.replace('.', '', regex=False)
                .str.replace('-', '', regex=False)
                .str.strip()
                )
df['Cliente'] = (df['Cliente']
                .str.replace('.', '', regex=False)
                .str.replace('-', '', regex=False)
                .str.strip()
                )

In [66]:
df['convenuto'] = df['convenuto'] .str.upper()
df['debitore'] = df['debitore'] .str.upper()
df['attore'] = df['attore'] .str.upper()
df['Cliente'] = df['Cliente'] .str.upper()

In [67]:
mask_attore = df['attore'].str.contains('\+')
mask_convenuto = df['convenuto'].str.contains('\+')

df.loc[mask_attore, 'attore'] = df.loc[mask_attore, 'attore'].str.split('\+').str[0].str.strip()
df.loc[mask_convenuto, 'convenuto'] = df.loc[mask_convenuto, 'convenuto'].str.split('\+').str[0].str.strip()

In [68]:
# df = pd.DataFrame(results.explode().tolist())
target_dir= f'Output/{str(today)}'
os.makedirs(target_dir, exist_ok=True)
df.drop(columns={'compila'}).to_excel(f'Output/{str(today)}/{str(today)}_output_{a}_{b}_v1.xlsx',index=False)


In [69]:
df = df[df['stato']!='-']
mask = df.apply(lambda row: (row['Cliente'] in row['attore']) 
                   or (row['attore'] in row['Cliente']) or (row['convenuto'] in row['debitore'])
                   or (row['debitore'] in row['convenuto']), axis=1)

df['match']='no'
df.loc[mask, 'match'] = 'si'

df['esito'] = '?'
df.loc[(df['match'] == 'si') & (df['compila'] == 'si'), 'esito'] = 'f24'
df.loc[(df['match'] == 'si') & (df['compila'] == 'no'), 'esito'] = 'esito ok'
df.loc[(df['match'] == 'no') & (df['compila'] == 'no'), 'esito'] = 'verifica dati'
df.loc[(df['match'] == 'no') & (df['compila'] == 'si'), 'esito'] = 'verifica dati / possibile f24'


df.to_excel(f'Output/{str(today)}/{str(today)}_esiti_positivi_{a}_{b}_v2.xlsx',index=False)

Uniamo tutti i file di output

In [ ]:
df = pd.DataFrame()
folder_path = f'Output/{str(today)}'
dataframes = []

for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx') and 'output' in filename:
        file_path = os.path.join(folder_path, filename)
        dataframes.append(pd.read_excel(file_path))

if dataframes:
    df = pd.concat(dataframes, ignore_index=True)

df = df.drop_duplicates()
df.to_excel(f'Output/{str(today)}/{str(today)}_output_totale.xlsx',index= False)
            

In [ ]:
df = pd.DataFrame()
folder_path = f'Output/{str(today)}'
dataframes = []

for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx') and 'esiti' in filename:
        file_path = os.path.join(folder_path, filename)
        dataframes.append(pd.read_excel(file_path))

if dataframes:
    df = pd.concat(dataframes, ignore_index=True)

df = df.drop_duplicates()
df.to_excel(f'Output/{str(today)}/{str(today)}_esiti_positivi.xlsx',index=False)